In [21]:
functions = {}

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def is_operator(s):
    op_list = ['+','-','*','%','//','<=']
    return s in op_list 

def is_function(s):
    return s in functions 

#print (is_function("print"))

In [27]:
class FunctionEvaluator: 
    def __init__(self,f_name,code_lines):
        self.argument_list = code_lines[0].split()

        self.lines = code_lines[1:]
        # skip empty lines while parsing
        #self.lines = [x.strip() for x in open(file_name).read().split('\n') if x!= '']
        
        #variable maps
        self.vars = {'output' : None}
        self.name = f_name
    
    def printlines(self):
        #print (self.lines)
        print(f"{self.name} -> {self.argument_list}")
        print("\n".join(self.lines))
    
    def print_state(self):
        print (self.argument_list)
        print (self.vars)
    
    #def addline(self,line):
    #    self.lines.append(line)

    #def printlines (self):
    #    print (self.lines)

    def evaluate(self, s):
        #print ("in eval",s)
        stack = []
        tokens = s.split()
        for token in tokens:
            if is_number(token): stack.append(int(token))
            elif is_function(token): 
                #print ("*******function call started", token)
                #print ("stack", stack)
                #print ("##",(functions[token]).argument_list)
                a_l = (functions[token])[0].split() 
                #print ("###",a_l)
                       
                arguments = []

                for _ in a_l:
                    #print ("XXXX",a)
                    arguments.append(stack.pop())

                #while stack:
                #    arguments.append(stack.pop())
                    
                ev = FunctionEvaluator( f_name = token,code_lines=functions[token])
                
                stack.append(ev.run(arguments))
            elif token.isidentifier(): stack.append(self.vars[token])
            else:
                rhs = stack.pop()
                lhs = stack.pop()
                match token: 
                    case "+":
                        stack.append(lhs+rhs)
                    case "-":
                        stack.append(lhs-rhs)
                    case "*":
                        stack.append(lhs*rhs)
                    case "%":
                        stack.append(lhs%rhs)
                    case "//":
                        stack.append(lhs//rhs)
                    case "<=": 
                        if lhs <= rhs : r=1
                        else: r=0
                        stack.append(r)

        return stack.pop()
    
    def run(self,arguments = None):
        # print (self.lines)
        #program counter
        if arguments != None:
            for var, value in zip(self.argument_list,reversed(arguments)):
                self.vars[var] = value

        pc = 0
        pc_stack = []

        while pc < len(self.lines):
            
            #print (pc)
          
            line = self.lines[pc]
            
            print(line)

            if line.startswith('while'):
                pc_stack.append(('while',pc))

                _, r_str = line.split(maxsplit=1)
                condition = self.evaluate(r_str)
                if condition == 1:
                    #print("good")
                    pc +=1

                else: 
                    
                    #find the matching end
                    while_counter = 1
                    while while_counter!=0: 
                        pc +=1
                        line = self.lines[pc]
                        if line.startswith('end'): while_counter -=1
                        elif line.startswith('while'): while_counter +=1
                    
                    pc_stack.pop()
                    pc+=1                      

            elif line.startswith('end'):
                #go back to while
                print (pc_stack)
                inst, pc = pc_stack.pop() 
            
            elif line.startswith('if'):
                _, r_str =line.split(maxsplit=1)
                condition = self.evaluate(r_str)

                if condition == 1:
                    pc += 1
                
                else: 
                    print ("if condition false")
                    if_counter = 0
                    #jump to the line after corresponding 'else'
                    while True:
                        pc += 1
                        line = self.lines[pc]
                        if line.startswith('if'): if_counter += 1
                        elif line.startswith('fi') and if_counter == 0: break 
                        elif line.startswith('fi'): if_counter -= 1
                        elif line.startswith('else') and if_counter == 0: 
                            break
                    pc += 1
            elif line.startswith('fi'): pc += 1

            elif line.startswith('else'):
                #jump to the line after corresponding 'fi'
                if_counter = 1
                while if_counter!=0:
                    pc+=1
                    line = self.lines[pc]
                    if line.startswith('if'): if_counter += 1
                    elif line.startswith('fi'): if_counter -= 1
                pc += 1
            elif line.startswith('break'):
                #find corresponding end
                while_counter = 1
                while while_counter !=0: 
                    pc += 1
                    line = self.lines[pc]
                    if line.startswith('while'): while_counter += 1
                    elif line.startswith('end'): while_counter -= 1
                pc +=1
            else: 
                lval, symbol, r_str = line.split(maxsplit=2)
                rval = self.evaluate(r_str)
                self.vars[lval] = rval
                pc+=1
        

        #print (self.vars)

        #if 'output' not in self.vars: return None
        return self.vars['output']
        
            #self.evaluate(line)
#ev =Evaluator("p5.code")
#ev.printlines()
#ev.run()

In [34]:

def load(file_name):
    lines = [x.strip() for x in open(file_name).read().split('\n') if x!= '' and not x.startswith("#")]
    func = "main"
    #ev = FunctionEvaluator(func,[])

    functions[func] = [""]
    for line in lines:
        if line.startswith("def"):
            _, f, params = line.split(maxsplit=2)
            func = f
            #fev = FunctionEvaluator(func, params.split())
            functions[func] = [params] 
        elif line.startswith('fed'):
            func = "main"
        else : 
            functions[func].append(line)



load("code/p6.code")
print (functions)
evl = FunctionEvaluator(f_name='main',code_lines=functions['main'])

evl.run()
#for f in functions:
    #(functions[f]).printlines()
    #print((functions[f]).name)
    #(functions[f]).print_state()

#print((functions["main"]).run())




#print (functions)


{'main': ['', 'x = 2', 'z = 4', 'output = 5 3 comb'], 'fact': ['n', 'r = 1', 'i = 1', 'while i n <=', 'r = r i *', 'i = i 1 +', 'end', 'output = r'], 'div': ['a b', 'output = a b //'], 'add': ['a b', 'output = a b +'], 'neg': ['a', 'output = 0 a -'], 'comb': ['n r', 'output = n fact r fact div n r neg add fact div']}
x = 2
z = 4
output = 5 3 comb
output = n fact r fact div n r neg add fact div
r = 1
i = 1
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
output = r
r = 1
i = 1
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while i n <=
output = r
output = a b //
output = 0 a -
output = a b +
r = 1
i = 1
while i n <=
r = r i *
i = i 1 +
end
[('while', 2)]
while

10

: 

In [ ]:
def near (p1, p2, delta =2):
    distv = p1-p1
    s_dist = np.inner(distv,distv)
    # dist = np.linalg.norm(p1 - p2)
    if s_dist <= delta*delta :
        return True
    else: 
        return False 

In [2]:
import numpy as np

a = np.array([2,3])
np.inner(a,a)

13